In [1]:
!pip install trtokenizer
!pip install TurkishStemmer
!pip install https://huggingface.co/turkish-nlp-suite/tr_core_news_trf/resolve/main/tr_core_news_trf-any-py3-none-any.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.2/849.2 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 26.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using ca

In [3]:
import pandas as pd
import re
import string
from trtokenizer.tr_tokenizer import SentenceTokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from snowballstemmer import stemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from TurkishStemmer import TurkishStemmer
import spacy
nlp = spacy.load("tr_core_news_trf")

# Importing as module.
import tr_core_news_trf
nlp = tr_core_news_trf.load()



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_excel('tweetData.xlsx')

In [5]:
df = df.dropna()

In [6]:
df.shape

(1976, 2)

In [7]:
def remove_special_characters(text):
    try:
      text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
      return text
    except:
        pass

def to_lowercase(text):
    try:
      return text.lower()
    except:
        pass

def remove_extra_spaces(text):
    try:
      return  ' '.join(text.split())
    except:
        pass

def remove_html_tags(text):
    try:
      clean = re.compile('<.*?>')
      return re.sub(clean, '', text)
    except:
        pass

def remove_urls(text):
    try:
      return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    except:
        pass

def remove_numbers(text):
    try:
      return re.sub(r'\d+', '', text)
    except:
        pass

def remove_punctuation(text):
    try:
      translator = str.maketrans('', '', string.punctuation)
      return text.translate(translator)
    except:
      pass

In [8]:
df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.Tweets.loc[15595:15599]

15595    kimmiş lan rahatsız olacak söylesin infaz edel...
15596                              i̇nstasina infaz edelim
15597    üstad latin abd ülkelerinde denediler direk ka...
15598    bu iki oevladının açık adreslerini verin infaz...
15599     kıbrısın tamamı alınmalı   ada diye bize yutt...
Name: Tweets, dtype: object

In [10]:
df.head()

,Tweets,Etiket
0,aynı bayrak aynı göğün altındayız biriz berabe...,1.0
1,çok istekli olan bu erkekler lütfen erkek çoc...,1.0
2,kurtlerle komşumuz alrabayiz beraberiz siz yal...,2.0
3,müslümanın seni beni ırkı olmaz dini olur kims...,1.0
4,soramazsına hoşgeldiniz bu gün bilge ile berab...,2.0


In [14]:


# Cümle ve kelime ayrıştırıcı işlevler
def trtokenizer_sentence(text):
    try:
        return sent_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Sentence tokenization error: {e}")
        return []

def trtokenizer_word(text):
    try:
        return word_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Word tokenization error: {e}")
        return []

# Stop kelimeleri temizleme işlevi
def remove_stopwords_nltk(words):
    stop_words = set(stopwords.words('turkish'))
    return [word for word in words if word.lower() not in stop_words]



def spacy_lemme(text):
    # Join the list of words into a single string
    text = ' '.join(text)
    doc = nlp(text)
    lemme = []
    for t in doc:
      lemme.append(t.lemma_)
    return ' '.join(lemme)

# Cümle ve kelime ayrıştırma işlemlerini DataFrame'e uygulama
df['sentences'] = df['Tweets'].apply(trtokenizer_sentence)

# Kelimeleri çıkarmak için her cümleyi kelimelere böler ve tüm kelimeleri düz bir liste olarak toplar
df['words'] = df['sentences'].apply(lambda sentences: [word for sentence in sentences for word in trtokenizer_word(sentence)])

# Stop kelimeleri temizleme işlemini ve birleştirme işlemini DataFrame'e uygulama
df['cleaned_words'] = df['words'].apply(remove_stopwords_nltk)
df['lemmatized_words'] = df['cleaned_words'].apply(spacy_lemme)

# Sonuçları yazdırma
print(df.head())




                                              Tweets  Etiket  \
0  aynı bayrak aynı göğün altındayız biriz berabe...     1.0   
1  çok istekli olan bu erkekler  lütfen erkek çoc...     1.0   
2  kurtlerle komşumuz alrabayiz beraberiz siz yal...     2.0   
3  müslümanın seni beni ırkı olmaz dini olur kims...     1.0   
4  soramazsına hoşgeldiniz bu gün bilge ile berab...     2.0   

                                           sentences  \
0  [aynı bayrak aynı göğün altındayız biriz berab...   
1  [çok istekli olan bu erkekler  lütfen erkek ço...   
2  [kurtlerle komşumuz alrabayiz beraberiz siz ya...   
3  [müslümanın seni beni ırkı olmaz dini olur kim...   
4  [soramazsına hoşgeldiniz bu gün bilge ile bera...   

                                               words  \
0  [aynı, bayrak, aynı, göğün, altındayız, biriz,...   
1  [çok, istekli, olan, bu, erkekler, lütfen, erk...   
2  [kurtlerle, komşumuz, alrabayiz, beraberiz, si...   
3  [müslümanın, seni, beni, ırkı, olmaz, dini, ol...  

In [15]:

# TfidfVectorizer ile metinleri vektörleştir
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(df['lemmatized_words'])
print(tfidf_vectors)


  (0, 149)	0.11755899121120444
  (0, 2186)	0.20403799129031985
  (0, 4966)	0.11870260706616925
  (0, 4807)	0.16943883424327486
  (0, 2083)	0.18870490362115586
  (0, 5116)	0.20951060715811903
  (0, 666)	0.237014251197528
  (0, 6035)	0.237014251197528
  (0, 2326)	0.16370406574568377
  (0, 661)	0.1664347005959182
  (0, 1741)	0.08911639876454933
  (0, 2773)	0.20951060715811903
  (0, 5052)	0.1832322877533567
  (0, 332)	0.15889117501286676
  (0, 75)	0.18870490362115586
  (0, 5806)	0.08815322675969457
  (0, 5921)	0.15572864371394773
  (0, 464)	0.21620854766056483
  (0, 3453)	0.2784365918570355
  (0, 3261)	0.16370406574568377
  (0, 792)	0.08198656181341892
  (0, 5305)	0.1511016129267542
  (0, 278)	0.1449226989350872
  (0, 721)	0.15197171110412033
  (0, 808)	0.224843694827283
  :	:
  (1974, 654)	0.13490080771503363
  (1974, 2873)	0.18155792681162608
  (1974, 1004)	0.15213707565928974
  (1974, 5419)	0.15213707565928974
  (1974, 1212)	0.09190191011580445
  (1974, 3417)	0.2521206744018542
  (1974,